<pre>
Copyright 2022-2025 Boris Shminke

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.
</pre>

In [16]:
# since both Jupyter and `isabelle-client` use `asyncio` we need to
# enable nested event loops. We don't need that when using
# `isabelle-client` from Python scripts outside Jupyter
import nest_asyncio

nest_asyncio.apply()

In [17]:
# first, we start Isabelle server
from isabelle_client import start_isabelle_server

server_info, isabelle_process = start_isabelle_server()

In [18]:
# server info is the same as printed by `isabelle server` command
print(server_info)

server "isabelle" = 127.0.0.1:46491 (password "ede07900-280a-4914-9320-82102aa815a1")



In [19]:
# a process object can be used for additional control
type(isabelle_process)

asyncio.subprocess.Process

In [20]:
isabelle_process

<Process 1043>

In [21]:
isabelle_process.pid

1043

In [22]:
# now we can create a client instance
from isabelle_client import get_isabelle_client

isabelle = get_isabelle_client(server_info)
isabelle

In [23]:
# the server's commands are implemented as client object methods
responses = isabelle.help()
responses

[IsabelleResponse(response_type='OK', response_body='{"isabelle_id":"29f2b8ff84f3","isabelle_name":"Isabelle2024"}', response_length=None),
 IsabelleResponse(response_type='OK', response_body='["cancel","echo","help","purge_theories","session_build","session_start","session_stop","shutdown","use_theories"]', response_length=118)]

In [24]:
# usually, the last server's response contains something interesting
responses[-1]

IsabelleResponse(response_type='OK', response_body='["cancel","echo","help","purge_theories","session_build","session_start","session_stop","shutdown","use_theories"]', response_length=118)

In [25]:
responses[-1].response_body

'["cancel","echo","help","purge_theories","session_build","session_start","session_stop","shutdown","use_theories"]'

In [26]:
# the response body is usually JSON formatted
import json

data = json.loads(responses[-1].response_body)
data

['cancel',
 'echo',
 'help',
 'purge_theories',
 'session_build',
 'session_start',
 'session_stop',
 'shutdown',
 'use_theories']

In [27]:
%%time
# let's do something more meaningful

session_id = isabelle.session_start()
session_id

CPU times: user 5.71 ms, sys: 791 μs, total: 6.5 ms
Wall time: 4.21 s


'927bcfd7-8a3a-4036-868a-182cd475d241'

In [28]:
# let's generate a dummy problem file in Isabelle format
with open("Problem.thy", "w") as problem_file:
    problem_text = """
theory Problem imports Main
begin
lemma "\<forall> x. \<exists> y. x = y"
sledgehammer
oops
end
    """
    problem_file.write(problem_text)

In [29]:
%%time
# now let's ask the server to solve the problem

responses = isabelle.use_theories(
    theories=["Problem"], master_dir=".", session_id=session_id
)
responses

CPU times: user 3.5 ms, sys: 922 μs, total: 4.42 ms
Wall time: 2.34 s


[IsabelleResponse(response_type='OK', response_body='{"isabelle_id":"29f2b8ff84f3","isabelle_name":"Isabelle2024"}', response_length=None),
 IsabelleResponse(response_type='OK', response_body='{"task":"4de9c47a-9ffb-4268-a074-aa58816d19d0"}', response_length=None),
 IsabelleResponse(response_type='NOTE', response_body='{"percentage":18,"task":"4de9c47a-9ffb-4268-a074-aa58816d19d0","message":"theory Draft.Problem 18%","kind":"writeln","session":"","theory":"Draft.Problem"}', response_length=161),
 IsabelleResponse(response_type='NOTE', response_body='{"percentage":99,"task":"4de9c47a-9ffb-4268-a074-aa58816d19d0","message":"theory Draft.Problem 99%","kind":"writeln","session":"","theory":"Draft.Problem"}', response_length=161),
 IsabelleResponse(response_type='NOTE', response_body='{"percentage":100,"task":"4de9c47a-9ffb-4268-a074-aa58816d19d0","message":"theory Draft.Problem 100%","kind":"writeln","session":"","theory":"Draft.Problem"}', response_length=163),
 IsabelleResponse(response_

In [30]:
# notice that important messages live inside "nodes" object
data = json.loads(responses[-1].response_body)
data

{'ok': True,
 'errors': [],
 'nodes': [{'messages': [{'kind': 'writeln',
     'message': 'Sledgehammering...',
     'pos': {'line': 5,
      'offset': 60,
      'end_offset': 72,
      'file': 'Problem.thy'}},
    {'kind': 'writeln',
     'message': 'cvc4 found a proof...',
     'pos': {'line': 5,
      'offset': 60,
      'end_offset': 72,
      'file': 'Problem.thy'}},
    {'kind': 'writeln',
     'message': 'verit found a proof...',
     'pos': {'line': 5,
      'offset': 60,
      'end_offset': 72,
      'file': 'Problem.thy'}},
    {'kind': 'writeln',
     'message': 'zipperposition found a proof...',
     'pos': {'line': 5,
      'offset': 60,
      'end_offset': 72,
      'file': 'Problem.thy'}},
    {'kind': 'writeln',
     'message': 'vampire found a proof...',
     'pos': {'line': 5,
      'offset': 60,
      'end_offset': 72,
      'file': 'Problem.thy'}},
    {'kind': 'writeln',
     'message': 'zipperposition found a proof...',
     'pos': {'line': 5,
      'offset': 60,
 

In [31]:
# the messages from `sledgehammer` have an easily parsable format
messages = [message["message"] for message in data["nodes"][0]["messages"]]
messages

['Sledgehammering...',
 'cvc4 found a proof...',
 'verit found a proof...',
 'zipperposition found a proof...',
 'vampire found a proof...',
 'zipperposition found a proof...',
 'verit: Try this: by simp (1 ms)',
 'cvc4: Duplicate proof',
 'zipperposition: Try this: by auto (1 ms)',
 'zipperposition: Try this: by force (1 ms)',
 'vampire: Duplicate proof',
 'spass found a proof...',
 'spass: Try this: by blast (1 ms)',
 'Done']

In [32]:
# we can parse `sledgehammer` suggestions using regular expressions
import re

proposed_solutions = [
    re.findall(".*: Try this: (.*) \(\d+\.?\d* m?s\)", message)
    for message in messages
]
proposed_solutions

[[],
 [],
 [],
 [],
 [],
 [],
 ['by simp'],
 [],
 ['by auto'],
 ['by force'],
 [],
 [],
 ['by blast'],
 []]

In [33]:
# in principle, we can value some solutions more
import random

solution = random.choice(
    [
        proposed_solution[0]
        for proposed_solution in proposed_solutions
        if proposed_solution
    ]
)
solution

'by simp'

In [34]:
# now let's write down our solution to a file
with open("Solution.thy", "w") as solution_file:
    solution_text = problem_text.replace(
        "sledgehammer\noops\n", f"{solution}\n"
    ).replace("theory Problem ", "theory Solution ")
    print(solution_text)
    solution_file.write(solution_text)


theory Solution imports Main
begin
lemma "\<forall> x. \<exists> y. x = y"
by simp
end
    


In [35]:
%%time
# and finally let's ask the server to check the solution

responses = isabelle.use_theories(
    theories=["Solution"], master_dir=".", session_id=session_id
)
responses

CPU times: user 1.67 ms, sys: 955 μs, total: 2.63 ms
Wall time: 2.01 s


[IsabelleResponse(response_type='OK', response_body='{"isabelle_id":"29f2b8ff84f3","isabelle_name":"Isabelle2024"}', response_length=None),
 IsabelleResponse(response_type='OK', response_body='{"task":"f7b9cbe1-2828-4103-8f79-86b0922c037b"}', response_length=None),
 IsabelleResponse(response_type='NOTE', response_body='{"percentage":99,"task":"f7b9cbe1-2828-4103-8f79-86b0922c037b","message":"theory Draft.Solution 99%","kind":"writeln","session":"","theory":"Draft.Solution"}', response_length=163),
 IsabelleResponse(response_type='FINISHED', response_body='{"ok":true,"errors":[],"nodes":[{"messages":[{"kind":"writeln","message":"theorem \\\\<forall>x. \\\\<exists>y. x = y","pos":{"line":5,"offset":61,"end_offset":63,"file":"Solution.thy"}}],"exports":[],"status":{"percentage":100,"unprocessed":0,"running":0,"finished":9,"failed":0,"total":9,"consolidated":true,"canceled":false,"ok":true,"warned":0},"theory_name":"Draft.Solution","node_name":"Solution.thy"}],"task":"f7b9cbe1-2828-4103-8f

In [36]:
# the response body doesn't contain any errors
json.loads(responses[-1].response_body)

{'ok': True,
 'errors': [],
 'nodes': [{'messages': [{'kind': 'writeln',
     'message': 'theorem \\<forall>x. \\<exists>y. x = y',
     'pos': {'line': 5,
      'offset': 61,
      'end_offset': 63,
      'file': 'Solution.thy'}}],
   'exports': [],
   'status': {'percentage': 100,
    'unprocessed': 0,
    'running': 0,
    'finished': 9,
    'failed': 0,
    'total': 9,
    'consolidated': True,
    'canceled': False,
    'ok': True,
    'warned': 0},
   'theory_name': 'Draft.Solution',
   'node_name': 'Solution.thy'}],
 'task': 'f7b9cbe1-2828-4103-8f79-86b0922c037b'}

In [37]:
# we can gracefully finish the session
isabelle.session_stop(session_id)

[IsabelleResponse(response_type='OK', response_body='{"isabelle_id":"29f2b8ff84f3","isabelle_name":"Isabelle2024"}', response_length=None),
 IsabelleResponse(response_type='OK', response_body='{"task":"be21f2f3-9053-4415-9e47-5d832bd39a9c"}', response_length=None),
 IsabelleResponse(response_type='FINISHED', response_body='{"ok":true,"return_code":0,"task":"be21f2f3-9053-4415-9e47-5d832bd39a9c"}', response_length=None)]

In [38]:
# and shut the server down
isabelle.shutdown()

[IsabelleResponse(response_type='OK', response_body='{"isabelle_id":"29f2b8ff84f3","isabelle_name":"Isabelle2024"}', response_length=None),
 IsabelleResponse(response_type='OK', response_body='', response_length=None)]